In [ ]:
import requests
import json
import pandas as pd
import scipy as sc
%matplotlib inline
from masterscrap import masterscrap

In [ ]:
conseil_dict = {
    "Ignazio Cassis" : 2664,
    "Doris Leuthard" : 2510,
    "Ueli Maurer" : 2310,
    "Johann Schneider-Ammann" : 2530,
    "Christoph Blocher" : 2017,
    "Guy Parmelin" : 2621,
    "Didier Burkhalter" : 2589,
    
}
nb_pages = 10

In [ ]:
def fetch_factions(id_):
    url = "http://ws-old.parlament.ch/factions/" + str(id_)
    print(url)
    params = {
        'format':'json'}
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
    }
    resp = requests.get(url=url, params=params, headers=headers)
    return resp

In [ ]:
r = fetch_factions(4)

In [ ]:
j = r.json()

In [ ]:
list_factions = [4, 2, 1, 6]

In [ ]:
total_list = {}
party_dict = {}
for id_ in list_factions:
    r = fetch_factions(id_)
    print (r.status_code)
    j = r.json()
    dict_ = {member['firstName'] + " " + member['lastName'] : member['number'] for member in j['members']}
    partyy = {member['number'] : id_ for member in j['members']}
    party_dict.update(partyy)
    total_list.update(dict_)

In [ ]:
len(total_list)

In [ ]:
nb_pages = 5
df = masterscrap(total_list , nb_pages)

In [ ]:
df

In [ ]:
df.to_pickle("parlement_votes_5pages.pkl")

In [ ]:
df = pd.read_pickle("parlement_votes_5pages.pkl")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_graph(graph, df):
    # extract nodes from graph
    nodes = set([n1 for n1, n2 in graph] + [n2 for n1, n2 in graph])

    # create networkx graph
    G=nx.Graph()
    G.position = {}

    # add nodes
    for node in nodes:
        G.add_node(node)

    # add edges
    for edge in graph:
        G.add_edge(edge[0], edge[1])

    
    # draw graph
    pos=nx.spring_layout(G)
    color_map = []
    ids = df.columns.values
    for node in G:
        if node in ids:
            color_map.append('blue')
        else: 
            color_map.append('green')  
    nx.draw(G, pos, node_color=color_map)

    # show graph
    plt.show()

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
# draw example
edges = []
for c in df:
    votes = df[c]
    edges += [(c, id_) for id_ in df.index.tolist() if votes.loc[id_] == 'Yes']

In [ ]:
# draw example
draw_graph(edges, df)

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
to_replace = {'Yes':2, 'No':1, 'P':0, 'EH':0, 'ES':0, 'NT':0}
clean_df = df.replace(to_replace)
clean_df = clean_df.fillna(0)
clean_df

## Sparse version

In [ ]:
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(clean_df)
H = model.components_

In [ ]:
clean_df.shape

In [ ]:
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(clean_df)
H = model.components_

In [ ]:
H.shape

In [ ]:
W.shape

In [ ]:
total_list['Jean-Luc Addor']

In [ ]:
plt.title("Politician Concept Mapping")
x = H[0,:]
y = H[1,:]
color_dict = {4:'chocolate', 2:'red', 1:'blue', 6:'green'}
plt.xlabel("Component 1 score")
plt.ylabel("Component 2 score")
j=0
for i,c in enumerate(total_list):
    print(H[0,:][i])
    plt.plot(H[0,:][i], H[1,:][i], color=color_dict[party_dict[total_list[c]]])
    #plt.annotate(c, (H[0,:][i], H[1,:][i]))
plt.show()

In [ ]:
plt.title("Vote Concept Mapping")
x = W[:,0]
y = W[:,1]
plt.scatter(x,y)
plt.xlabel("Component 1 score")
plt.ylabel("Component 2 score")
plt.show()

In [ ]:
model = NMF(n_components=7, init='random', random_state=0)
W = model.fit_transform(clean_df)
H = model.components_

In [ ]:
from sklearn.metrics import mean_squared_error

nR = np.dot(W,H)
mean_squared_error(clean_df.values.flatten(), list(nR.flatten()))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
W_embedded = TSNE(n_components=2).fit_transform(W)
H_embedded = TSNE(n_components=2).fit_transform(H)

In [ ]:
H_embedded.shape, W_embedded.shape

In [ ]:
plt.title("Politician Concept Mapping")
x = H_embedded[:,0]
y = H_embedded[:,1]
plt.scatter(x,y)
plt.xlabel("Component 1 score")
plt.ylabel("Component 2 score")
for i,c in enumerate(conseil_dict):
    plt.annotate(c, (H_embedded[:,0][i], H_embedded[:,1][i]))
plt.show()

In [ ]:
plt.title("Vote Concept Mapping")
x = W_embedded[:,0]
y = W_embedded[:,1]
plt.scatter(x,y)
plt.xlabel("Component 1 score")
plt.ylabel("Component 2 score")
plt.show()

In [ ]:
#SVD
import numpy as np
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(clean_df - np.mean(clean_df), k = 5)

In [ ]:
U.shape, Vt.shape

In [ ]:
plt.title("Politician Concept Mapping")
x = Vt[0,:]
y = Vt[1,:]
plt.scatter(x,y)
plt.xlabel("Component 1 score")
plt.ylabel("Component 2 score")
for i,c in enumerate(total_list):
    plt.annotate(c, (Vt[0,:][i], Vt[1,:][i]))
plt.show()